In [19]:
import gc
import os
import pathlib
import time

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import StratifiedKFold

In [3]:
%matplotlib inline

In [4]:
train_df = pd.read_csv('../input/train/train.csv')

In [5]:
acoustic_data_df = train_df[train_df.columns.drop('time_to_failure')].astype(np.int32)

In [6]:
time_to_failure_df = train_df[train_df.columns.drop('acoustic_data')].astype(np.float32)

In [8]:
del train_df
gc.collect()

15

In [9]:
acoustic_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 629145480 entries, 0 to 629145479
Data columns (total 1 columns):
acoustic_data    int32
dtypes: int32(1)
memory usage: 2.3 GB


In [10]:
time_to_failure_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 629145480 entries, 0 to 629145479
Data columns (total 1 columns):
time_to_failure    float32
dtypes: float32(1)
memory usage: 2.3 GB


In [11]:
train_values, holdout_test_values, train_target_values, holdout_test_target_values = train_test_split(
    acoustic_data_df.values,
    time_to_failure_df.values,
    test_size=0.25,
    random_state=0
)

In [14]:
linear_regression_model = LinearRegression(n_jobs=10)

In [15]:
start_time = time.time()
linear_regression_model.fit(train_values, train_target_values)
print("Total training model time: {}".format((time.time() - start_time) / 60))

Total training model time: -0.3664133350054423


In [16]:
prediction = linear_regression_model.predict(holdout_test_values)

In [17]:
print("mean_squared_error: ", mean_squared_error(holdout_test_target_values, prediction))

mean_squared_error:  13.489476553031523


In [20]:
print("mean_absolute_error: ", mean_absolute_error(holdout_test_target_values, prediction))

mean_absolute_error:  3.0482380705479266


In [21]:
svm_regression_model = SVR()

In [ ]:
start_time = time.time()
svm_regression_model.fit(train_values, train_target_values)
print("Total training model time: {}".format((time.time() - start_time) / 60))